In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import pandas as pd

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def get_multiclass_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=1)
    stars = torch.argmax(scores, dim=1).item() + 1  # Star rating 1-5
    return stars, scores.squeeze().tolist()

# Apply to each bio
df_bio = pd.read_csv("C:/Users/Andy/Documents/VSCODE/NFL Player Bio and Grade Data.csv") # Change path
df_bio.head()

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\Users\Andy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Andy\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

,Player,Player Bio,Player Grades,Draft Class,Link
0,Jadeveon Clowney,A physical specimen with a rare size-speed com...,7.5,2014,https://www.nfl.com/prospects/jadeveon-clowney...
1,Sammy Watkins,A legitimate No. 1-caliber receiver who steppe...,7.1,2014,https://www.nfl.com/prospects/sammy-watkins/32...
2,Anthony Barr,"A highly disruptive, athletic specimen with th...",7.0,2014,https://www.nfl.com/prospects/anthony-barr/320...
3,Khalil Mack,A havoc-wreaking rush linebacker with the burs...,7.0,2014,https://www.nfl.com/prospects/khalil-mack/3200...
4,Jake Matthews,"Smart, tough, versatile franchise left tackle ...",7.0,2014,https://www.nfl.com/prospects/jake-matthews/32...


In [2]:
sentiment_results = df_bio['Player Bio'].apply(get_multiclass_sentiment)
df_bio['star_rating'] = sentiment_results.apply(lambda x: x[0])
df_bio[['score_1', 'score_2', 'score_3', 'score_4', 'score_5']] = pd.DataFrame(sentiment_results.apply(lambda x: x[1]).tolist(), index=df_bio.index)
df_bio.head()

,Player,Player Bio,Player Grades,Draft Class,Link,star_rating,score_1,score_2,score_3,score_4,score_5
0,Jadeveon Clowney,A physical specimen with a rare size-speed com...,7.5,2014,https://www.nfl.com/prospects/jadeveon-clowney...,4,0.020313,0.057753,0.121158,0.459664,0.341111
1,Sammy Watkins,A legitimate No. 1-caliber receiver who steppe...,7.1,2014,https://www.nfl.com/prospects/sammy-watkins/32...,5,0.004270,0.003489,0.004970,0.073340,0.913931
2,Anthony Barr,"A highly disruptive, athletic specimen with th...",7.0,2014,https://www.nfl.com/prospects/anthony-barr/320...,5,0.089131,0.116260,0.085209,0.235079,0.474321
3,Khalil Mack,A havoc-wreaking rush linebacker with the burs...,7.0,2014,https://www.nfl.com/prospects/khalil-mack/3200...,4,0.063653,0.158930,0.193473,0.381666,0.202278
4,Jake Matthews,"Smart, tough, versatile franchise left tackle ...",7.0,2014,https://www.nfl.com/prospects/jake-matthews/32...,5,0.000751,0.001039,0.006316,0.272827,0.719067


In [3]:
df_bio.to_csv("Player Bio and Grade with Sentiment Score.csv")